####  CNN based on ResNet50 ####

In [1]:
import sys, json, os

In [2]:
import pandas as pd
import numpy as np

In [36]:
import skimage.io
import matplotlib.pyplot as plt

In [4]:
#from tensorflow.python.keras.applications import ResNet50
import tensorflow
from tensorflow import keras
tensorflow.__version__

'2.2.0'

In [5]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


In [7]:
from tensorflow.keras.models import save_model

In [8]:
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam

In [13]:
from panda_bvv_config import *


In [10]:
model_name = os.path.join(note_path, 'model_panda.h5')
checkpoint_name = os.path.join(note_path, 'model_panda_check')

In [ ]:
#on tf2:
#256x256 - bsize=32, 180/40 steps/epoch
#512x512 - bsize=20, 250/60 steps/epoch
#30 epochs are enough

In [11]:
bsize = 32
num_epochs = 30
image_sizey = 512
image_sizex = 512
s_per_epoch= 180
val_steps = 40

In [12]:
temp_arr = []
for path, subdirs, files in os.walk(train_cnn):
    for name in files:
        temp_arr.append(name)
num_train_files = len(temp_arr)
temp_arr = []
for path, subdirs, files in os.walk(valid_cnn):
    for name in files:
        temp_arr.append(name)
num_valid_files = len(temp_arr)

In [14]:
class_weights  = {
                 0: 1.0,
                 1: 1.0848,
                 2: 2.1534,
                 4: 2.3155,
                 3: 2.3285,
                 5: 2.3627
                 }

In [15]:
#s_per_epoch= num_train_files//bsize+1
#val_steps = num_valid_files//bsize+1


In [41]:
# #that's OK:
# from tensorflow_addons.metrics import  CohenKappa
# kappa_keras = CohenKappa(num_classes=6,weightage='quadratic')

In [22]:
data_generator = ImageDataGenerator(rescale=1 / 255.0,
                                    rotation_range=20,
                                    zoom_range=0.05,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.05,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    #fill_mode="nearest"
                                   )

In [23]:
train_generator = data_generator.flow_from_directory(
        train_cnn,
        target_size=(image_sizey, image_sizex),
        color_mode="rgb",
        batch_size= bsize,
        shuffle = True,
        class_mode='categorical')



Found 0 images belonging to 0 classes.


In [24]:
v_generator = ImageDataGenerator(rescale=1 / 255.0)

In [27]:
validation_generator = v_generator.flow_from_directory(
        valid_cnn,
        target_size=(image_sizey, image_sizex),
        color_mode="rgb",
        batch_size = bsize,
        shuffle = False,
        class_mode='categorical')


Found 0 images belonging to 0 classes.


In [28]:
callbacks_list = [
#         keras.callbacks.EarlyStopping(
#         # Stop training when `val_loss` is no longer improving
#         monitor='val_loss',
#         mode = 'min',
#         # "no longer improving" being defined as "no better than 1e-2 less"
#         min_delta=1e-2,
#         # "no longer improving" being further defined as "for at least 2 epochs"
#         patience=10,
#         verbose=1,
#         restore_best_weights = True
#         ),
        
        keras.callbacks.ModelCheckpoint(
        filepath= checkpoint_name +".{epoch:02d}.h5",
        monitor='val_loss',
        mode = 'auto',
        save_weights_only = False,
        save_freq = 'epoch',
        save_best_only=False
        ),
    
        keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=.1,
        patience=5,
        verbose=1,
        min_lr=1e-7
        )
]

In [29]:
resnet = ResNet50(include_top=False, pooling='avg', weights='imagenet')

In [30]:
new_model = Sequential()
new_model.add(resnet)
new_model.add(Flatten())
new_model.add(Dense(512, activation='relu'))
new_model.add(Dense(num_classes, activation='softmax'))

In [31]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 24,639,878
Trainable params: 24,586,758
Non-trainable params: 53,120
_________________________________________________________________


In [18]:
# Say not to train first layer (ResNet) model. It is already trained
#new_model.layers[0].trainable = False

In [33]:
new_model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=1e-3),
                  metrics=['categorical_accuracy'] #, kappa_keras])

In [21]:
history = new_model.fit_generator(
        train_generator,
        steps_per_epoch=s_per_epoch,
        class_weight=class_weights,
        epochs = num_epochs,
        callbacks=callbacks_list,
        validation_data=validation_generator,
        validation_steps=val_steps,
        verbose=1)

Epoch 1/2
20/20 [==============================] - 629s 31s/step - loss: 1.8496 - acc: 0.2828 - val_loss: 1.9098 - val_acc: 0.2562
Epoch 2/2
20/20 [==============================] - 593s 30s/step - loss: 1.6811 - acc: 0.3234 - val_loss: 1.8938 - val_acc: 0.2406


In [34]:
history_file = 'history_respca512.json'
dict_to_save = {}
for k, v in history.history.items():
    dict_to_save.update({k: [np.format_float_positional(x) for x in history.history[k]]})
with open(history_file, 'w') as file:
    json.dump(dict_to_save, file)



NameError: name 'history' is not defined

In [37]:
plt.style.use("ggplot")
plt.figure()

plt.plot(np.arange(0, num_epochs), history.history['loss'], label='loss')
plt.plot(np.arange(0, num_epochs), history.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, num_epochs),history.history['categorical_accuracy'], label='cat. accuracy')
plt.plot(np.arange(0, num_epochs),history.history['val_categorical_accuracy'], label='val_accuracy')
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_loss.png")

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>

In [2]:
# plt.style.use("ggplot")
# plt.figure()

# plt.plot(np.arange(0, num_epochs), history.history['kappa_keras'], label='cohen_kappa')
# plt.plot(np.arange(0, num_epochs), history.history['val_kappa_keras'], label='val_cohen_kappa')
# plt.title("Training Kappa on Dataset")
# plt.xlabel("Epoch #")
# plt.ylabel("Kappa train/val")
# plt.legend(loc="lower left")
# plt.savefig("plot_kappa.png")

In [23]:
new_model.save(os.path.join(base_path, model_name))

In [1]:
import os
module_name = 'respca_simple_fit'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)